Execute the following block of code by selecting it and clicking ``ctrl + enter`` to create an ``NvidiaRacecar`` class.  

In [1]:
from jetracer.nvidia_racecar import NvidiaRacecar
import time

car = NvidiaRacecar()

The ``NvidiaRacecar`` implements the ``Racecar`` class, so it has two attributes ``throttle`` and ``steering``. 

We can assign values in the range ``[-1, 1]`` to these attributes.  Execute the following to set the steering to 0.4.

> If the car does not respond, it may still be in ``manual`` mode.  Flip the manual override switch on the RC transmitter.

## Test the control units

In [2]:
# Control parameters for testing
steering_range = 0.7  # Maximum steering value (range: -steering_range to +steering_range)
step = 10  # Number of steps in each direction (total steps = 2*step + 1)
delay = 0.1  # Delay between each step in seconds

for steering_value in [round(x * (steering_range / step), 2) for x in range(-step, step + 1)]:
    car.steering = steering_value
    print(f"Set steering to {steering_value}")
    time.sleep(delay)

# Reset steering to center
car.steering = 0.0
print("Steering reset to 0.0")


Set steering to -0.7
Set steering to -0.63
Set steering to -0.56
Set steering to -0.49
Set steering to -0.42
Set steering to -0.35
Set steering to -0.28
Set steering to -0.21
Set steering to -0.14
Set steering to -0.07
Set steering to 0.0
Set steering to 0.07
Set steering to 0.14
Set steering to 0.21
Set steering to 0.28
Set steering to 0.35
Set steering to 0.42
Set steering to 0.49
Set steering to 0.56
Set steering to 0.63
Set steering to 0.7
Steering reset to 0.0


In [6]:
# Test double-click reverse sequence
print("🧪 Testing ESC Double-Click Reverse...")
print("⚠️  Make sure the car has plenty of space!")

# 1. Move forward first
print("\n1️⃣ Moving forward...")
car.throttle = 0.2
time.sleep(2)

# 2. First reverse command (brake only)
print("\n2️⃣ First reverse command (should brake, not reverse)...")
car.throttle = -0.3
time.sleep(2)

# 3. Return to neutral (ESC ready for reverse)
print("\n3️⃣ Returning to neutral (ESC preparing for reverse)...")
car.throttle = 0.0
time.sleep(1)

# 4. Second reverse command (actual reverse)
print("\n4️⃣ Second reverse command (should actually reverse now)...")
car.throttle = -0.2
time.sleep(2)

# 5. Stop
print("\n5️⃣ Stopping...")
car.throttle = 0.0
print("✅ Test complete!")

🧪 Testing ESC Double-Click Reverse...
⚠️  Make sure the car has plenty of space!

1️⃣ Moving forward...
🚗 FORWARD: throttle=0.200 → output=0.060

2️⃣ First reverse command (should brake, not reverse)...
🛑 BRAKE (1st reverse): throttle=-0.300 → output=-0.340

3️⃣ Returning to neutral (ESC preparing for reverse)...
⚠️  ESC READY FOR REVERSE (returned to neutral after brake)
⏸️  NEUTRAL: output=-0.100

4️⃣ Second reverse command (should actually reverse now)...
⬅️  REVERSE (2nd reverse): throttle=-0.200 → output=-0.260

5️⃣ Stopping...
⏸️  NEUTRAL: output=-0.100
✅ Test complete!


In [5]:
# Test parameters
throttle_range = 0.5  # Reduced for safety
step = 8  # Fewer steps for safer testing
delay = 0.8  # Longer delay to observe direction

for throttle_value in [round(x * (throttle_range / step), 2) for x in range(-step, step + 1)]:
    direction = "BACKWARD" if throttle_value < 0 else "FORWARD" if throttle_value > 0 else "STOP"
    print(f"Setting throttle: {throttle_value:+.2f} → Expected: {direction}")
    car.throttle = throttle_value
    time.sleep(delay)

# Reset throttle to stop
car.throttle = 0.0
print("\n✅ Throttle reset to 0.0 - Car stopped")


Setting throttle: -0.50 → Expected: BACKWARD
🛑 BRAKE (1st reverse): throttle=-0.500 → output=-0.500
Setting throttle: -0.44 → Expected: BACKWARD
🛑 BRAKE (fallback): throttle=-0.440 → output=-0.452


KeyboardInterrupt: 

In [ ]:
# Make sure the car is stopped
car.throttle = 0.0

The ``NvidiaRacecar`` class has two values ``steering_gain`` and ``steering_bias`` that can be used to calibrate the steering.

We can view the default values by executing the cells below.

In [ ]:
print(car.steering_gain)

In [ ]:
print(car.steering_offset)

The final steering value is computed using the equation

$y = a \times x + b$

Where,

* $a$ is ``car.steering_gain``
* $b$ is ``car.steering_offset``
* $x$ is ``car.steering``
* $y$ is the value written to the motor driver

You can adjust these values calibrate the car so that setting a value of ``0`` moves forward, and setting a value of ``1`` goes fully right, and ``-1`` fully left.

To set the throttle of the car to ``0.2``, you can call the following.

> Give JetRacer lots of space to move, and be ready on the manual override, JetRacer is *fast*

In [ ]:
car.throttle = -0.0

The throttle also has a gain value that could be used to control the speed response.  The throttle output is computed as

$y = a \times x$

Where,

* $a$ is ``car.throttle_gain``
* $x$ is ``car.throttle``
* $y$ is the value written to the speed controller

Execute the following to print the default gain

In [ ]:
print(car.throttle_gain)

Set the following to limit the throttle to half

In [ ]:
car.throttle_gain = 0.5

## 🚗 ESC Double-Click Reverse Behavior

**Important**: The JetRacer Pro uses an ESC (Electronic Speed Controller) that requires a "double-click" method for reversing:

### Forward Movement:
- **Positive throttle values** (e.g., 0.3) make the car move **forward** ✅

### Reverse Movement (Double-Click Required):
1. **First negative throttle** → **BRAKE ONLY** 🛑 (no reverse yet)
2. **Return to neutral** (0.0) → **ESC READY** ⚠️ 
3. **Second negative throttle** → **ACTUAL REVERSE** ⬅️

### Example Reverse Sequence:
```python
# Step 1: Apply negative throttle (this will BRAKE, not reverse)
car.throttle = -0.3  # 🛑 BRAKE
time.sleep(1)

# Step 2: Return to neutral 
car.throttle = 0.0   # ⚠️ ESC READY FOR REVERSE
time.sleep(0.5)

# Step 3: Apply negative throttle again (this will REVERSE)
car.throttle = -0.3  # ⬅️ ACTUAL REVERSE
```

### Troubleshooting:
- If your car moves in the wrong direction, use `car.auto_calibrate_throttle_direction()` to fix it automatically.
- Watch the console output for helpful state indicators: 🚗 FORWARD, 🛑 BRAKE, ⬅️ REVERSE, ⏸️ NEUTRAL

In [ ]:
# Test double-click reverse sequence
print("🧪 Testing ESC Double-Click Reverse...")
print("⚠️  Make sure the car has plenty of space!")

# 1. Move forward first
print("\n1️⃣ Moving forward...")
car.throttle = 0.2
time.sleep(2)

# 2. First reverse command (brake only)
print("\n2️⃣ First reverse command (should brake, not reverse)...")
car.throttle = -0.3
time.sleep(2)

# 3. Return to neutral (ESC ready for reverse)
print("\n3️⃣ Returning to neutral (ESC preparing for reverse)...")
car.throttle = 0.0
time.sleep(1)

# 4. Second reverse command (actual reverse)
print("\n4️⃣ Second reverse command (should actually reverse now)...")
car.throttle = -0.3
time.sleep(2)

# 5. Stop
print("\n5️⃣ Stopping...")
car.throttle = 0.0
print("✅ Test complete!")


That's it for this notebook!